In [ ]:
#install.packages("scales")
library(readxl)
library(ggplot2)
library("FactoMineR")
library(tidyverse) #data Manipulation
library(cluster) #Clustering algorithms
library(factoextra)#Clustering Algorithms & Visualization
library(gridExtra)
#install.packages("dendextend")
#install.packages("ggplot2")
#install.packages("rlang")
library(dendextend) # For fancy deprograms
library(rlang) # clustering cutting
library("np") #npreg: you may need to install it
library(PerformanceAnalytics)
library(psych)
#install.packages('fmsb')
library(fmsb) #For Spider Plots
#install.packages('countrycode')
library(countrycode)
#install.packages(c('hrbrthemes','viridis'))
#library(hrbrthemes)
#library(viridis)
#install.packages('GGally')
library(GGally)
library(dplyr)
library(ggmap)
library(leaflet)
library(htmltools)
library(dplyr) #getMap()
library(rgeos)
#install.packages('rworldmap')
library(rworldmap) #joinCountryData2Map
library(grid)
library(tidyr)
#install.packages(c("cowplot", "googleway", "ggplot2", "ggrepel", "ggspatial", "sf", "rnaturalearth", "rnaturalearthdata"))
# library("sf")
library("rnaturalearth")
library("rnaturalearthdata")
#install.packages('writexl')
library(writexl)
library("RColorBrewer")
#install.packages('reshape2')
library(reshape2)
require(rgdal)
#install.packages('mapproj')
require(mapproj)


In [ ]:
df.wa.fa <- read_excel("../../results/df-seven-scores.xlsx",sheet=1)
dfsimple <- read_excel("../../results/dfsimple.xlsx",sheet=1)

#Giving unique names for the factors
df <- df.wa.fa[,c(2:8)]
head(df)
df <- df %>% 
  rename(
      'Far Well' = ML1,
      'Vended'= ML2,
      'Piped Indoors'= ML3,
      'Nearby Improved'= ML4,
      'Piped Outdoors' = ML5,
      'Far Spring' = ML6,
      'Nearby Surface'= ML7
     )

#resizing graphs accordingly
options(repr.plot.width=10, repr.plot.height=10)                  
par(mar=c(1,1,1,1))

                 
pdf(file = "../../docs/manuscript/pdf-image/factor-cor.pdf"
  ,
  width     = 10,
  height    = 10 )

chart.Correlation(df, histogram=TRUE, pch=19 , tl.cex = .7 )

dev.off()



In [ ]:
# Create data: Water Accessibility Aggregate
df.wa.aggregate <- aggregate(df, list(clusters=dfsimple$clusters), mean)
df.wa.avg <- df.wa.aggregate[,1:8]
df.wa.avg

# Set1 easy to see '#E41A1C','#377EB8','#4DAF4A','#984EA3','#FF7F00'
brewer.pal(n = 3, name = "Set2") # Best color scheme is Set2 for color blind friendly 

#barplots to recognize the distribution
centroid <- apply(df.wa.aggregate, 1, function(x) sqrt(sum(x^2)) ) #How close is each country to the centroid of the cluster
#resizing graphs accordingly
options(repr.plot.width=10, repr.plot.height=7)                  
old.par <- par(mar = c(0, 0, 0, 0))
par(old.par)
          

pdf(file = "../../docs/manuscript/pdf-image/centroid.pdf"
  ,
  width     = 10,
  height    = 10 )
                  
bp <- barplot(centroid,
        main = "", xlab = "Clusters", ylab = "fitness of Centroid",
        col = c('#66c2a5','#fc8d62', '#8da0cb'),
        ylim = c(0, 10), names.arg = c("Decentralized" , "Hybrid", "Centralized"))
dev.off()       
                  


In [ ]:
#resizing graphs accordingly
options(repr.plot.width=17, repr.plot.height=10)                  
        
#Typlology of the three distniguished typology 
pdf(file = "../../docs/manuscript/pdf-image/typology.pdf"
  ,
  width     = 15,
  height    = 10 )
                  
barplot(as.matrix(df.wa.avg[,c(2:8)]),
        beside = TRUE,
        main = "", xlab = "Water variables", ylab = "Factor Weight",
        col = c('#66c2a5','#fc8d62', '#8da0cb'),
        legend = c("Decentralized" , "Hybrid", "Centralized"), 
        args.legend = list(title = "Clusters", x = "topright", cex = .7), ylim = c(0, 1))
dev.off()       
                  


In [ ]:
# Development of the spider plot of the typology understanding
centroiddf.wa <- apply(df, 1, function(x) sqrt(sum(x^2)) ) #How close is each country to the cen-troid of the cluster

data.spider <-   df.wa.avg[,c(2:8)]    
rownames(data.spider) <- c("Decentralized" , "Hybrid", "Centralized")   
                       
data.spider <- rbind(rep(1,1) , rep(0,1) , df.wa.avg[,c(2:8)])
                  
head(data.spider)
colnames(data.spider) <- c(                 
   "Far Well",
"Far Spring",
"Piped Indoors",
    'Piped Outdoors',
    'Vended',
    "Nearby Surface",
    'Nearby Improved')  
                                     
options(repr.plot.width=20, repr.plot.height=20)
par(mar = c(1,1,1,1))
par(cex=1.4)
colors_in=c('#66c2a5','#fc8d62', '#8da0cb')
                       

pdf(file = "../../docs/manuscript/pdf-image/spider-plot.pdf" ,width     = 12,height    = 12 )
                       
# The default radar chart for Typology
r.c. <- radarchart(data.spider,
           #custom polygon
           pcol= c('#66c2a5','#fc8d62', '#8da0cb') ,
           plwd=4 , plty=1,
           #custom the grid
           cglcol="grey", cglty=1, axislabcol="grey", caxislabels= seq(0,1,.2), cglwd=0.8, axistype = 4,
           title = '',
           vlcex= 1.3,
           calcex = 1.3 ,
                  ) 
legend(x=.8, y=1.25, legend = c("Decentralized" , "Hybrid", "Centralized"), bty = "n", pch=20 , col=colors_in , text.col = "black", cex=1.2, pt.cex=3)                       
 dev.off()

# Box Plot of Water Accessibility Variables (Violin Plots)

In [ ]:
# Giving unique typology names
df.wa.aggregate <- aggregate(df, list(clusters=dfsimple$clusters), mean)
df.wa.aggregate <- df.wa.aggregate[,2:8]
rownames(df.wa.aggregate) <- c("Decentralized" , "Hybrid", "Centralized")
data_wa_one <- df.wa.aggregate[,1:7]
df.wa.aggregate <- as.factor(df.wa.aggregate[,])

In [ ]:
df.wa.fa <- read_excel("../../results/df-seven-scores.xlsx",sheet=1)
#giving unique names
df.wa.fa <- df.wa.fa %>% 
  rename(
      'Far Well' = ML1,
      'Vended'= ML2,
      'Piped Indoors'= ML3,
      'Nearby Improved'= ML4,
      'Piped Outdoors' = ML5,
      'Far Spring' = ML6,
      'Nearby Surface'= ML7
     )

df = read_excel('../../results/df-fa-seven-cluster-rank.xlsx') #change name of file to sth descriptive (e.g. country-clusters)
dfc = df[, c(1,11)] # select only country name and cluster number
# Giving unique typlogy names
dfc <- dfc%>%
  mutate(clusters=case_when(
    .$clusters=="1" ~ "Decentralized",
    .$clusters=="2" ~ "Hybrid",
    .$clusters=="3" ~ "Centralized",
      ))

df <- df.wa.fa[,c(1:8)]
dfv <- merge(x = dfc,
              y = df,
              by = c("Country"))

# Convert the variable dose from a numeric to a factor variable
dfv$clusters <- as.factor(dfv$clusters)
weight.m <- melt(dfv, id="clusters", 
                   measure=c(    
        
        'Far Well',
        'Nearby Improved',
        'Nearby Surface',
        'Vended' ,
      'Piped Indoors' ,
      'Piped Outdoors' ,
      'Far Spring' ), 
                   variable.name="Factor", value.name="Weight")
#Sizing
options(repr.plot.width=17, repr.plot.height=7)
par(mar = c(1,1,1,1))
par(cex=5)

# Basic violin plot for all seven factors
p <- ggplot(data = weight.m, aes(x=Factor, y=Weight, fill = clusters)) + 
  geom_violin(trim=FALSE) +
    theme_minimal() + 
    stat_summary(fun.data=mean_sdl, 
                 geom="pointrange", color="black") +
labs(title="",x="Factors", y = "Factor Scores") +
 scale_fill_manual(
      name = 'Typology', 
  labels = c("Decentralized" , "Hybrid", "Centralized"),
    values=c('#8da0cb','#66c2a5','#fc8d62')) +
  facet_wrap(~ clusters , nrow=1) +
  coord_flip() +
    theme(text = element_text(size=20))


pdf(file = "../../docs/manuscript/pdf-image/violin.pdf"
  ,
  width     = 15,
  height    = 10 )

p 
dev.off()




In [ ]:
dfw <- read_excel("../../results/df-water-access.xlsx",sheet=1)
dfw['clusters'] <- dfv$clusters


dfw <- dfw %>% 
  rename(
    'Piped (Dwelling)' = phom ,
     'Piped (Yard)' = pipy,
     'Public Tap' = ptap,
      'Borehole'=bore,
      'Protected Well'=pwel,
     'Protected Spring' =pspr,
      'Rain'= rain,
      'Unprotected Well'=uwel,
      'Unprotected Spring'=uspr,
     'Truck' =truc,
      'Tanker Cart'=ctan,
      'Bottled'=bott,
     'Other' =othw,
     'Surface' =surw,
      'T<30min'=tles,
      'T>30min' =tmor,
     'On-Premises' =watp,
    )

head(dfw)

agg.m <- melt(dfw, id="clusters", 
                   measure=c(
'Piped (Dwelling)', 'Piped (Yard)','Public Tap','Borehole','Protected Well','Protected Spring','Rain','Unprotected Well','Unprotected Spring'
                       ,'Truck','Tanker Cart','Bottled','Other','Surface','T<30min','T>30min','On-Premises'
                   ), 
                   variable.name="WA", value.name="Aggregation")
#Sizing
options(repr.plot.width=17, repr.plot.height=17)
par(mar = c(1,1,1,1))
par(cex=5)

# Violin plot for all 17 water accessibility variables 
w <- ggplot(data = agg.m, aes(x= WA, y= Aggregation, fill = Aggregation, color = clusters)) + 
  geom_violin(trim=FALSE) +
    theme_minimal() + 
    stat_summary(fun.data=mean_sdl, 
                 geom="pointrange") +
    labs(title="",x="", y = "% households") +
    scale_color_manual(name = 'Typology', 
    labels = c("Decentralized" , "Hybrid", "Centralized"),
    values=c('#8da0cb','#66c2a5','#fc8d62')) +
  facet_wrap(~ clusters , ncol=3, as.table = TRUE) +
  coord_flip() +
    theme(text = element_text(size=20))

pdf(file = "../../docs/manuscript/pdf-image/violin-17-wa-clusters.pdf"
  ,
  width     = 15,
  height    = 15 )

w

dev.off()




In [ ]:
# Box plot of the 17 water accesisibility variables
b<-ggplot(agg.m, aes(x=WA, y= Aggregation, fill = clusters)) +
    geom_boxplot() +
    theme_minimal() +
    scale_fill_manual(name = 'Typology', 
    labels = c("Decentralized" , "Hybrid", "Centralized"),
    values=c( '#8da0cb','#66c2a5','#fc8d62')) +
    facet_wrap(~ clusters , ncol=3, as.table = TRUE) +
    coord_flip() +
    labs(title="",x="", y = "% households", element_text(size=17)) +
    theme(legend.position="bottom") +
    theme(text = element_text(size=17)) + #Font size
    theme(legend.title = element_blank(), legend.position = "none") + #No need of legend 
    theme(axis.text.x = element_text(size=17),
          axis.text.y = element_text(size=17)) #Adjusting the tick sizes


pdf(file = "../../docs/manuscript/pdf-image/boxplot.pdf"
  ,
  width     = 14,
  height    = 10 )

par(mar = c(1, 1, 1, 20), cex.axis = 1, cex.lab = .7) 

b

dev.off()

# Ranking

In [ ]:
df.wa <- read_excel("../../results/df-seven-scores.xlsx",sheet=1)
# clean df at next save (2 cluster columns - only 1 needed; etc) #df also needs to be only water accessibility variable
df.wa["norm"] <- apply(df.wa[2:8], 1, function(x) sqrt(sum(x^2)) ) #assuming columns 5:10 are the water accessibility variables
# Ideally, df should just be the countries and the water access. variables ONLY. This, way, the code should then be:
# df["norm"] <- apply(df, 1, function(x) sqrt(sum(x^2)) )          
df.wa["distToCentroid"] = 0 # initialize           
df.wa["clusters"] <- as.factor(dfsimple$clusters)
#Turn into numeric to find centroids             
# assuming centroiddf.wa is your vector of cluster centroids
# copied this from your earlier notebook
df.wa.aggregate <- aggregate(df.wa[,c(2:8)], list(clusters=dfsimple$clusters), mean)
df.wa.aggregate <- df.wa.aggregate[,c(1:7)]
centroiddf.wa <- apply(df.wa.aggregate, 1, function(x) sqrt(sum(x^2)) ) #How close is each country to the cen-troid of the cluster                        
df <- df.wa            
# Then here, you compute the distance to centroid cluster by cluster
df[df$clusters == 1, "distToCentroid"] = abs(df[df$clusters == 1, "norm"] - centroiddf.wa[1])
df[df$clusters == 2, "distToCentroid"] = abs(df[df$clusters == 2, "norm"] - centroiddf.wa[2])
df[df$clusters == 3, "distToCentroid"] = abs(df[df$clusters == 3, "norm"] - centroiddf.wa[3])
#df[df$clusters == 4, "distToCentroid"] = abs(df[df$clusters == 4, "norm"] - centroiddf.wa[4])
#df[df$clusters == 5, "distToCentroid"] = abs(df[df$clusters == 5, "norm"] - centroiddf.wa[5])
#df[df$clusters == 6, "distToCentroid"] = abs(df[df$clusters == 6, "norm"] - centroiddf.wa[6])                       
write_xlsx(df , '../../results/df-fa-seven-cluster-rank.xlsx')
                       
# You can then sort each subset by the size of "distToCentroid"
df1 = df %>% 
  arrange_at("Country", desc) %>%
   arrange_at("distToCentroid") %>%
    arrange_at("clusters") %>%
  select(1, 10, 11)
cluster.one <- subset(df1, clusters == 1 )
cluster.two <- subset(df1, clusters == 2 )
cluster.three <- subset(df1, clusters == 3 )
cluster.four <- subset(df1, clusters == 4 )
cluster.five <- subset(df1, clusters == 5 )
#cluster.six <- subset(df1, clusters == 6 )
summary(cluster.one)            
cluster.one %>% 
  arrange_at("Country", desc) %>%
   arrange_at("distToCentroid") %>%
    arrange_at("clusters") %>%
  select(1:3)


summary(cluster.two)                         
cluster.two %>% 
  arrange_at("Country", desc) %>%
   arrange_at("distToCentroid") %>%
    arrange_at("clusters") %>%
  select(1:3)


summary(cluster.three) 
cluster.three %>% 
  arrange_at("Country", desc) %>%
   arrange_at("distToCentroid") %>%
    arrange_at("clusters") %>%
  select(1:3)

                       
# cluster.four %>% 
#   arrange_at("Country", desc) %>%
#    arrange_at("distToCentroid") %>%
#    arrange_at("clusters") %>%
#  select(1:3)
#cluster.five %>% 
# arrange_at("Country", desc) %>%
#  arrange_at("distToCentroid") %>%
# arrange_at("clusters") %>%
# select(1:3)
#cluster.six %>% 
  #arrange_at("Country", desc) %>%
  # arrange_at("distToCentroid") %>%
  # arrange_at("clusters") %>%
# select(1:3)

# World Map

https://stackoverflow.com/questions/18323832/plot-small-region-of-a-large-polygon-map-in-ggplot2

In [ ]:
df = read_excel('../../results/df-fa-seven-cluster-rank.xlsx') #change name of file to sth descriptive (e.g. country-clusters)
df = df[, c(1,11)] # select only country name and cluster number
colnames(df) = c('region', 'value') # change column names for ggplot

# This is to convert country names in df to match those in the worldMap dataframe
coords <- joinCountryData2Map(df, joinCode = "NAME", nameJoinColumn = "region")
coords = data.frame(coords)
colnames(coords) = tolower(colnames(coords))
colnames(coords)
coords = drop_na(coords, value)
coords$name
setdiff(df$region,coords$name)

# Correct the country names in DF
df[df=='Timor-Leste'] = "East Timor"
df[df=='Central African Republic'] = "Central African Rep."
df[df=='Cote d\'Ivoire'] = "Ivory Coast"
df[df=='Dominican Republic'] = "Dominican Rep."
df[df=='Kyrgyz Republic'] = "Kyrgyzstan"
df[df=='Eswatini'] = "Swaziland"
df[df=='Congo'] = "Congo (Brazzaville)"
df[df=='Congo Democratic Republic'] = "Congo (Kinshasa)"

# Get world map data (lat/long)
countrynames = df$region
worldMap <- getMap()
country_indices <- which(worldMap$NAME%in%countrynames)

# Get coordinates for all countries to plot base map
allCoords <- lapply(seq(1,243), function(i){
  dfn <- data.frame(worldMap@polygons[[i]]@Polygons[[1]]@coords)
  dfn$region =as.character(worldMap$NAME[i])
  colnames(dfn) <- list("long", "lat", "region")
  return(dfn)
})
allCoords <- do.call("rbind", allCoords)

# Get coordinates strictly for those in water accessibility dataset
waterCoords <- lapply(country_indices, function(i){
  dfn <- data.frame(worldMap@polygons[[i]]@Polygons[[1]]@coords)
  dfn$region =as.character(worldMap$NAME[i])
  colnames(dfn) <- list("long", "lat", "region")
  return(dfn)
})

waterCoords <- do.call("rbind", waterCoords)
waterCoords$value <- df$value[match(waterCoords$region,df$region)]
waterCoords$value <- factor(waterCoords$value)

# Plot
options(repr.plot.width=10, repr.plot.height=7)
par(mar = c(1,1,1,1))
par(cex=1)
m <- ggplot() +
    geom_sf(color = "black", fill= 'antiquewhite') +
    xlab("") + ylab("") +
    geom_polygon(data= allCoords, mapping = aes(x =long, y=lat, group = region), color="grey", fill=NA) + 
    geom_polygon( data = waterCoords, mapping = aes(x = long, y = lat, group = region, fill=value), size=.3,color="black") +
    expand_limits(x = waterCoords$long, y = waterCoords$lat)  + 
    scale_fill_brewer(palette='Set2', name="Water Accessibility Typologies", na.value = "white", 
                      labels = c("Decentralized" , "Hybrid", "Centralized") )  + 
    theme(
  # Hide panel borders and remove grid lines
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  # Change axis line
  axis.line = element_line(colour = "white")
  ) + 

    theme(legend.position = "top",
        axis.text.x = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks = element_blank(),
        rect = element_blank(), 
        axis.text=element_text(size=20),
        axis.title=element_text(size=20)) +
   theme(legend.key.size = unit(1, 'cm'), #change legend key size
        legend.key.height = unit(1, 'cm'), #change legend key height
        legend.key.width = unit(1, 'cm'),
        legend.title = element_text(size=20,face=),
        legend.text = element_text(size=20)) +
scale_x_continuous(limits = c(-145, 190)) +
scale_y_continuous(limits = c(-55, 100)) +  


#     #guides(fill = guide_colorbar(barwidth = 10, barheight = .5))

pdf(file = "../../docs/manuscript/pdf-image/cluster-map.pdf"
  ,
  width     = 15,
  height    = 7 )

m + coord_map(xlim = c(-85, 145),ylim = c(-35, 50))

dev.off()

View a single RColorBrewer palette by specifying its name
display.brewer.pal(n = 8, name = 'Set1')
Set1 easy to see'#E41A1C','#377EB8','#4DAF4A','#984EA3','#FF7F00'
brewer.pal(n = 5, name = "Set1") # Best color scheme is Set1 